In [1]:
import os

import pandas as pd
import numpy as np

from UsageFetch import refreshUsageData
from dotenv import load_dotenv
from nixtlats import TimeGPT

load_dotenv()

True

In [2]:
timegpt = TimeGPT(token=os.getenv("TIMEGPT_TOKEN"))
timegpt.validate_token()

INFO:nixtlats.timegpt:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [3]:
refreshUsageData()

No new usage data detected. Token usage remains the same.

= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

          API CALLS:  47   ||   TOKENS:  229260   ||   SPENT:  380.02          

= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 


In [ ]:
bdSample2 = pd.read_csv("billing_data_sample2.csv")

bdSample2

In [ ]:
bdSample2["TimePeriod"] = pd.to_datetime(bdSample2["TimePeriod"]).dt.tz_localize(None)

In [ ]:
bdSample2

In [ ]:
bdSample2Temp = bdSample2.drop(columns=["Tenant"])

In [ ]:
bdSample2Temp

In [ ]:
bdSample2Temp.to_csv("billing_data_sample2_temp.csv", index=False)

In [ ]:
data = pd.read_csv("billing_data_sample2_temp.csv")
data

In [ ]:
anomalies = timegpt.detect_anomalies(data, time_col="TimePeriod", target_col="ProratedQuantity")

In [ ]:
refreshUsageData()

In [ ]:
anomalies

In [ ]:
anomalies[anomalies["anomaly"] == 1]

In [ ]:
timegpt.plot(data, anomalies, time_col="TimePeriod", target_col="ProratedQuantity")

In [ ]:
level = [99, 90, 99.99]
forecast = timegpt.forecast(
    data,
    h=1,
    time_col="TimePeriod",
    target_col="ProratedQuantity",
    freq="D",
    add_history=True,
    level=[99, 90, 99.99],
)

forecast = forecast.merge(data)
for lv in level:
    forecast[f"anomaly_{lv}"] = (
        forecast["ProratedQuantity"] > forecast[f"TimeGPT-hi-{lv}"]
    ) | (forecast["ProratedQuantity"] < forecast[f"TimeGPT-lo-{lv}"])
    forecast[f"anomaly_{lv}"] = forecast[f"anomaly_{lv}"].astype(np.int64)

In [ ]:
refreshUsageData()

In [ ]:
pd.testing.assert_series_equal(
    forecast["anomaly_99"],
    anomalies.merge(data)["anomaly"],
    check_index=False,
    check_names=False
)

In [ ]:
anomalies = timegpt.detect_anomalies(
    data, time_col="TimePeriod", target_col="ProratedQuantity", freq="D", level=90
)
timegpt.plot(data, anomalies, time_col="TimePeriod", target_col="ProratedQuantity")

In [ ]:
refreshUsageData()

In [ ]:
pd.testing.assert_series_equal(
    forecast["anomaly_90"],
    anomalies.merge(data)["anomaly"],
    check_index=False,
    check_names=False,
)

In [ ]:
anomalies = timegpt.detect_anomalies(
    data, time_col="TimePeriod", target_col="ProratedQuantity", freq="D", level=99.99
)
timegpt.plot(data, anomalies, time_col="TimePeriod", target_col="ProratedQuantity")

In [ ]:
refreshUsageData()

In [ ]:
pd.testing.assert_series_equal(
    forecast["anomaly_99.99"],
    anomalies.merge(data)["anomaly"],
    check_index=False,
    check_names=False,
)

In [ ]:
level = [99, 90, 99.99]
forecastWithDF = timegpt.forecast(
    data,
    h=1,
    time_col="TimePeriod",
    target_col="ProratedQuantity",
    freq="D",
    add_history=True,
    date_features=True,
    level=[99, 90, 99.99],
)

forecastWithDF = forecastWithDF.merge(data)
for lv in level:
    forecastWithDF[f"anomaly_{lv}"] = (
        forecastWithDF["ProratedQuantity"] > forecastWithDF[f"TimeGPT-hi-{lv}"]
    ) | (forecastWithDF["ProratedQuantity"] < forecastWithDF[f"TimeGPT-lo-{lv}"])
    forecastWithDF[f"anomaly_{lv}"] = forecastWithDF[f"anomaly_{lv}"].astype(np.int64)

In [ ]:
refreshUsageData()

In [ ]:
anomaliesWithDF = timegpt.detect_anomalies(
    data,
    time_col="TimePeriod",
    target_col="ProratedQuantity",
    freq="D",
    date_features=True,
    level=99.99,
)
timegpt.plot(
    data, anomaliesWithDF, time_col="TimePeriod", target_col="ProratedQuantity"
)

In [ ]:
refreshUsageData()

In [ ]:
pd.testing.assert_series_equal(
    forecastWithDF["TimeGPT-lo-99.99"],
    anomaliesWithDF.merge(data)["TimeGPT-lo-99.99"],
    check_index=False,
    check_names=False,
    rtol=1,
    atol=1,
)